### 1. Load packages, open dataset, load model

Open the AI-ready dataframe that includes the series column for prediction. Load the tuned model from the previous notebook.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from pycaret.regression import *

In [14]:
df = pd.read_csv('../data/ai_ready/middle_timeseries.csv')

In [3]:
model = load_model('randomforest')

Transformation Pipeline and Model Successfully Loaded


In [ ]:
df['date'] = pd.to_datetime(df['time'])  # Ensure date is datetime type
df = df.set_index('date').resample('6D').mean()  # Resample to every 6 days frequency
df = df.interpolate()  # Interpolate missing values


### 2. Visualize test data

The data were split into training and test data. Here we visualize the predictions for the training data and predictions for the test set (highlighted in light gray).

In [27]:
# Ensure the 'time' column is in datetime format
df['time'] = pd.to_datetime(df['time'])

# Generate predictions and use the original time column
predictions = predict_model(model, data=df)
predictions['time'] = df['time']  # Ensure this matches your data

# Ensure the lengths match
predictions = predictions.iloc[:len(df)]

# Plot
fig = px.line(predictions, x='time', y=["velocity", "prediction_label"], template='plotly_dark')
fig.add_vrect(x0="2021-06-01", x1="2024-04-18", fillcolor="grey", opacity=0.25, line_width=0)
fig.show()

### 3. Predict future values

The model is now trained and we can predict into the future. Let's start with values 4 years out at a frequency of 6 days.

Here we create a dataset with time and series values, like our original dataframe. The model will fill in the velocity prediction values.

In [35]:
future_dates = pd.date_range(start = '2024-04-18', end = '2028-04-18', freq = '6D')

future_df = pd.DataFrame()

future_df['time'] = future_dates
future_df['series'] = np.arange(145,(145+len(future_dates)))

future_df.head()

,time,series
0,2024-04-18,145
1,2024-04-24,146
2,2024-04-30,147
3,2024-05-06,148
4,2024-05-12,149


Make predictions on the future values.

In [36]:
predictions_future = predict_model(model, data=future_df)
predictions_future.head()

,time,series,prediction_label
0,2024-04-18,145,846.870675
1,2024-04-24,146,841.662524
2,2024-04-30,147,841.353200
3,2024-05-06,148,847.316025
4,2024-05-12,149,850.571864


Plot the original data and the future predictions.

In [38]:
concat_df = pd.concat([df,predictions_future], axis=0)
concat_df_i = pd.date_range(start='2015-01-30', end = '2028-04-24', freq = '6D')
concat_df.set_index(concat_df_i, inplace=True)

fig = px.line(concat_df, x=concat_df.index, y=["velocity", "prediction_label"], template = 'plotly_dark')
fig.show()

Overall, the predictions look fairly accurate, with some seasonal spikes during the melt season. The predictions even capture the subtle two spikes in velocity that we have been observing in recent years. Like mentioned earlier, the predictions did not fully capture the intensity of spikes but still were able to capture their general trend.

We can do the same predictions for the near-terminus and upstream points:

In [ ]:
df = pd.read_csv('../data/ai_ready/')

Let's compare all points on the same plot: